# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
####LocalTest#####
TFIDF_max_features = 10

In [6]:
import os
import json

data_file = "/content/data"
train_file = os.path.join(data_file, 'train-claims.json')
dev_file = os.path.join(data_file, 'dev-claims.json')
evidence_file = os.path.join(data_file, 'evidence.json')
test_file = os.path.join(data_file, 'test-claims-unlabelled.json')

with open(train_file, 'r') as f:
    tr_claims = json.load(f)

with open(dev_file, 'r') as f:
    dev_claims = json.load(f)

with open(evidence_file, 'r') as f:
    evidences = json.load(f)

with open(test_file, 'r') as f:
    test_claims = json.load(f)

print("Train claims:", len(tr_claims))
print("Dev claims:", len(dev_claims))   
print("Evidences:", len(evidences))
print("Test claims:", len(test_claims))

FileNotFoundError: [Errno 2] No such file or directory: '/content/data\\train-claims.json'

In [ ]:
# ####Local test#####
# import os
# import json

# train_file = "data/train-claims.json"
# with open(train_file, 'r') as f:
#     tr_claims = json.load(f)
    

# print("Train claims:", len(tr_claims))
# id = list(tr_claims.keys())[0]
# print("Train claim id:", id)
# tr_claim_test = tr_claims[id]
# tr_claim_test

Train claims: 1228
Train claim id: claim-1937


{'claim_text': 'Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.',
 'claim_label': 'DISPUTED',
 'evidences': ['evidence-442946', 'evidence-1194317', 'evidence-12171']}

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')

lemmatizer = WordNetLemmatizer()
def word_tokenize_and_lemmatize(text):
    def lemmatize_word(token):
        token = lemmatizer.lemmatize(token, pos='v')
        token = lemmatizer.lemmatize(token, pos='n') if token != token else token
        return token
    
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    lemmed_tokens = [lemmatize_word(token) for token in tokens]
    return lemmed_tokens

[nltk_data] Downloading package stopwords to C:\Users\Salist
[nltk_data]     desk2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Salist
[nltk_data]     desk2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# ####Local test#####
# word_tokenize_and_lemmatize(tr_claim_test['claim_text'])

['scientific',
 'evidence',
 'pollutant',
 'higher',
 'concentrations',
 'actually',
 'help',
 'ecosystems',
 'support',
 'plant',
 'animal',
 'life']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import joblib

evidence_ids = evidences.keys()
evidence_text_list = [evidences[eid] for eid in evidence_ids]
evidence_vectorizer = TfidfVectorizer(tokenizer=word_tokenize_and_lemmatize, max_features=5000)
evidence_vectors = evidence_vectorizer.fit_transform(evidence_text_list)

sparse.save_npz("evidence_vectors.npz", evidence_vectors)
joblib.dump(evidence_vectorizer, "evidence_vectorizer.pkl")

NameError: name 'evidences' is not defined

In [13]:
from scipy import sparse
import joblib
# sparse.save_npz("evidence_vectors.npz", evidence_vectors)
evidence_vectors = sparse.load_npz("evidence_vectors.npz")
evidence_vectorizer = joblib.load("evidence_vectorizer.pkl")

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*